In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib qt

In [2]:
import sys
sys.path.append('..')
import neural_ode.NeuralODE

In [3]:
tf.keras.backend.set_floatx('float64')

In [12]:
model = keras.Sequential(
    [
        keras.Input(shape=(2,)),
        layers.Dense(2, name="layer1"),
    ]
)


In [13]:
n_ode = neural_ode.NeuralODE.NeuralODE(model, 2)

In [ ]:
n_ode.pretrain()

In [6]:
# Generate solution

In [7]:
#
N_n = int(2)
c = 0.1
k = 4.0
def oscilator(t, y):
    return np.array([y[1], -c*y[1]-k*y[0]])
t_final = 20.0
n_eval = int(500)
t_span = np.array([0.0, t_final])
y0 = np.array([1.0, 0.0])

In [8]:
sol = solve_ivp(oscilator, t_span, y0, t_eval=np.linspace(0, t_final, num=n_eval))

In [9]:
plt.plot(sol.t, sol.y[0,:])

In [14]:
# transform to tensorflow
t_span_tf = tf.constant(t_span)
y0_tf = tf.constant(y0, dtype=tf.float64)
t_target = tf.constant(sol.t)
y_target = tf.constant(np.transpose(sol.y) )

In [15]:
sol1 = n_ode.forward_solve(t_target, y_target[0,:])
fig = plt.figure()
ax = plt.gca()
ax.plot(t_target.numpy(), y_target[:,0].numpy())
ax.plot(sol1['t'].numpy(), sol1['y'][:,0].numpy())

In [13]:
# adjoint method

In [16]:
loss, dl_dy, a = n_ode.adjoint_method(t_target[0:3], y_target[0:3,:])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [15]:
# fit

In [19]:
n_epoch = 40
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_fold=5, adjoint_method=False)

--- Epoch #1 ---
Total loss of epoch: 0.35424490831792355
Elapsed time for epoch: 2.2613649368286133
--- Epoch #2 ---
Total loss of epoch: 0.3028665566816926
Elapsed time for epoch: 2.1240234375
--- Epoch #3 ---
Total loss of epoch: 0.12583754118531942
Elapsed time for epoch: 2.0518605709075928
--- Epoch #4 ---
Total loss of epoch: 0.20920523954555392
Elapsed time for epoch: 2.8077049255371094
--- Epoch #5 ---
Total loss of epoch: 0.11579417996108532
Elapsed time for epoch: 2.500103712081909
--- Epoch #6 ---
Total loss of epoch: 0.14598148223012686
Elapsed time for epoch: 2.057159662246704
--- Epoch #7 ---
Total loss of epoch: 0.10314867692068219
Elapsed time for epoch: 2.0066030025482178
--- Epoch #8 ---
Total loss of epoch: 0.0536558935418725
Elapsed time for epoch: 2.3182601928710938
--- Epoch #9 ---
Total loss of epoch: 0.054390063625760376
Elapsed time for epoch: 2.382230520248413
--- Epoch #10 ---
Total loss of epoch: 0.03490475320722908
Elapsed time for epoch: 2.0779080390930176

In [ ]:
# Check derivatives

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c], [0.0, 1.0]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss, dl_dp = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)
#loss, dL_dy, a = n_ode.adjoint_method(t_target, y_target, x_external=f_ext_interp)
#dl_dp = a[2:]

In [ ]:
dp = 0.00001
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c+dp], [0.0, 1.0]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss2, dl_dp2 = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)

In [ ]:
(loss2-loss)/dp

In [ ]:
dl_dp